In [35]:
import pandas as pd

In [36]:
p = "mean of diseased pixels in red plane, mean of diseased pixels in green plane, mean of diseased pixels in blue plane, mean of background pixels in red plane, mean of background pixels in green plane, mean of background pixels in blue plane, mean of boundary region pixels of disease in red plane, mean of boundary region pixels of disease in green plane, mean of boundary region pixels of disease in blue plane, mean of core region pixels of disease in red plane, mean of core region pixels of disease in green plane, mean of core region pixels of disease in blue plane, standard deviation of diseased pixels in red plane, standard deviation of diseased pixels in green plane, standard deviation of diseased pixels in blue plane, standard deviation of background pixels in red plane, standard deviation of background pixels in green plane, standard deviation of background pixels in blue plane, standard deviation of boundary region pixels of disease in red plane, standard deviation of boundary region pixels of disease in green plane, standard deviation of boundary region pixels of disease in blue plane, standard deviation of core region pixels of disease in red plane, standard deviation of core region pixels of disease in green plane, standard deviation of core region pixels of disease in blue plane."

In [37]:
q = p.lower().split(",")

In [38]:
d = {'feature_of_data':q}
df = pd.DataFrame(d)

In [39]:
df

,feature_of_data
0,mean of diseased pixels in red plane
1,mean of diseased pixels in green plane
2,mean of diseased pixels in blue plane
3,mean of background pixels in red plane
4,mean of background pixels in green plane
5,mean of background pixels in blue plane
6,mean of boundary region pixels of disease in ...
7,mean of boundary region pixels of disease in ...
8,mean of boundary region pixels of disease in ...
9,mean of core region pixels of disease in red ...


In [40]:
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
PATH = os.getcwd()
# Define data path
data_path = PATH + '/Datasets'
data_dir_list = os.listdir(data_path)
print(data_dir_list)

['Bacterial leaf blight', 'Brown spot', 'Leaf smut']


In [41]:
import cv2
import numpy as np

def image_masking(img):
    BLUR = 21
    CANNY_THRESH_1 = 10
    CANNY_THRESH_2 = 200
    MASK_DILATE_ITER = 10
    MASK_ERODE_ITER = 10
    MASK_COLOR = (0.0,0.0,0.0) # In BGR format
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    edges = cv2.Canny(gray, CANNY_THRESH_1, CANNY_THRESH_2)
    edges = cv2.dilate(edges, None)
    edges = cv2.erode(edges, None)
    contour_info = []
    contours, __ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    for c in contours:
        contour_info.append((c, cv2.isContourConvex(c), cv2.contourArea(c),))
    contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
    
    max_contour = contour_info[0]
    mask = np.zeros(edges.shape)
    cv2.fillConvexPoly(mask, max_contour[0], (255))
    mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
    mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
    mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)
    
    mask_stack = np.dstack([mask]*3)
    mask_stack  = mask_stack.astype('float32') / 255.0
    img = img.astype('float32') / 255.0
    masked = (mask_stack * img) + ((1-mask_stack) * MASK_COLOR)
    masked = (masked * 255).astype('uint8')
    resized_mask= cv2.resize(masked,(256,256),interpolation = cv2.INTER_AREA)
    #cv2.imshow("reshaped_img",resized_mask)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return resized_mask

#if __name__ == '__main__':
#    filepath = "DSC_0373.JPG"
#    image_masking(filepath)

In [42]:
num_classes = 3

labels_name={'Bacterial leaf blight':0,'Brown spot':1,'Leaf smut':2}

img_data_list=[]
labels_list = []

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loading the images of dataset-'+'{}\n'.format(dataset))
    label = labels_name[dataset]
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        img_data_list.append(input_img)
        labels_list.append(label)

img_data = np.array(img_data_list)
print (img_data.shape)

Loading the images of dataset-Bacterial leaf blight

Loading the images of dataset-Brown spot

Loading the images of dataset-Leaf smut

(120,)


In [43]:
reshaped_img = []
for i in range(120):
    reshaped_img.append(cv2.resize(img_data[i],(256,256)))

In [44]:
rgb = []
for i in range(120):
    rgb.append(cv2.cvtColor(reshaped_img[i],cv2.COLOR_BGR2RGB))
hsv = []
canny = []
for i in range(120):
    hsv.append(cv2.cvtColor(rgb[i],cv2.COLOR_RGB2HSV))
    canny.append(cv2.Canny(rgb[i],100,200))
#for i in range(120):
#    cv2.imshow("rgb_of_image",rgb[i])
#    cv2.imshow("hsv_of_image",hsv[i])
#    cv2.imshow("canny",canny[i])
#    cv2.waitKey(0)
#cv2.destroyAllWindows()

In [45]:
R1 = []
G1 = []
B1 = []
for i in range(120):
    R,G,B = cv2.split(rgb[i])
    R1.append(R)
    G1.append(G)
    B1.append(B)

In [46]:
#core mean
sp_b = []
sp_g = []
sp_r = []
for i in range(120):
    b,g,r  = cv2.split(image_masking(reshaped_img[i]))
    sp_b.append(b.mean())
    sp_g.append(g.mean())
    sp_r.append(r.mean())

In [47]:
#core std
std_sp_b = []
std_sp_g = []
std_sp_r = []
for i in range(120):
    b,g,r  = cv2.split(image_masking(reshaped_img[i]))
    std_sp_b.append(b.std())
    std_sp_g.append(g.std())
    std_sp_r.append(r.std())

In [48]:
#mean boundary region pixel
canny_R = []
canny_G = []
canny_B = []
for i in range(120):
    canny_R.append(cv2.Canny(R1[i],100,200))
    canny_G.append(cv2.Canny(G1[i],100,200))
    canny_B.append(cv2.Canny(B1[i],100,200))

In [49]:
#mean boundary region pixel
mean_canny_R = []
mean_canny_G = []
mean_canny_B = []
for i in range(120):
    mean_canny_R.append(canny_R[i].mean())
    mean_canny_G.append(canny_G[i].mean())
    mean_canny_B.append(canny_B[i].mean())

In [50]:
#std boundary region pixel
std_canny_R = []
std_canny_G = []
std_canny_B = []
for i in range(120):
    std_canny_R.append(canny_R[i].std())
    std_canny_G.append(canny_G[i].std())
    std_canny_B.append(canny_B[i].std())

In [51]:
#mean disised pixel
mean_desised_pixel_R = []
mean_desised_pixel_G = []
mean_desised_pixel_B = []
for i in range(120):
    mean_desised_pixel_R.append(R1[i].mean())
    mean_desised_pixel_G.append(G1[i].mean())
    mean_desised_pixel_B.append(B1[i].mean())

In [52]:
#std disised pixel
std_desised_pixel_R = []
std_desised_pixel_G = []
std_desised_pixel_B = []
for i in range(120):
    std_desised_pixel_R.append(R1[i].std())
    std_desised_pixel_G.append(G1[i].std())
    std_desised_pixel_B.append(B1[i].std())

In [53]:
#mean back-ground pixel
back_gr_r = []
back_gr_g = []
back_gr_b = []
for i in range(120):
    ret,thresh_r=cv2.threshold(R1[i],50,255,cv2.THRESH_BINARY)
    ret,thresh_g=cv2.threshold(G1[i],50,255,cv2.THRESH_BINARY)
    ret,thresh_b=cv2.threshold(B1[i],50,255,cv2.THRESH_BINARY)
    back_gr_r.append(thresh_r.mean())
    back_gr_g.append(thresh_g.mean())
    back_gr_b.append(thresh_b.mean())

In [54]:
#std back-ground pixel
std_back_gr_r = []
std_back_gr_g = []
std_back_gr_b = []
for i in range(120):
    ret,thresh_r=cv2.threshold(R1[i],50,255,cv2.THRESH_BINARY)
    ret,thresh_g=cv2.threshold(G1[i],50,255,cv2.THRESH_BINARY)
    ret,thresh_b=cv2.threshold(B1[i],50,255,cv2.THRESH_BINARY)
    std_back_gr_r.append(thresh_r.std())
    std_back_gr_g.append(thresh_g.std())
    std_back_gr_b.append(thresh_b.std())

In [55]:
p = {"mean of diseased pixels in red plane":mean_desised_pixel_R,"mean of diseased pixels in green plane":mean_desised_pixel_G,"mean of diseased pixels in blue plane":mean_desised_pixel_B,"mean of background pixels in red plane":back_gr_r,"mean of background pixels in green plane":back_gr_g,"mean of background pixels in blue plane":back_gr_b,"mean of boundary region pixels of disease in red plane":mean_canny_R,"mean of boundary region pixels of disease in green plane":mean_canny_G,"mean of boundary region pixels of disease in blue plane":mean_canny_B,"mean of core region pixels of disease in red plane":sp_r,"mean of core region pixels of disease in green plane":sp_g,"mean of core region pixels of disease in blue plane":sp_b,"standard deviation of diseased pixels in red plane":std_desised_pixel_R,"standard deviation of diseased pixels in green plane":std_desised_pixel_G,"standard deviation of diseased pixels in blue plane":std_desised_pixel_B,"standard deviation of background pixels in red plane":std_back_gr_r,"standard deviation of background pixels in green plane":std_back_gr_g,"standard deviation of background pixels in blue plane":std_back_gr_b,"standard deviation of boundary region pixels of disease in red plane":std_canny_R,"standard deviation of boundary region pixels of disease in green plane":std_canny_G,"standard deviation of boundary region pixels of disease in blue plane":std_canny_B,"standard deviation of core region pixels of disease in red plane":std_sp_r,"standard deviation of core region pixels of disease in green plane":std_sp_g,"standard deviation of core region pixels of disease in blue plane":std_sp_b}

In [56]:
df = pd.DataFrame(p)

In [57]:
df

,mean of diseased pixels in red plane,mean of diseased pixels in green plane,mean of diseased pixels in blue plane,mean of background pixels in red plane,mean of background pixels in green plane,mean of background pixels in blue plane,mean of boundary region pixels of disease in red plane,mean of boundary region pixels of disease in green plane,mean of boundary region pixels of disease in blue plane,mean of core region pixels of disease in red plane,...,standard deviation of diseased pixels in blue plane,standard deviation of background pixels in red plane,standard deviation of background pixels in green plane,standard deviation of background pixels in blue plane,standard deviation of boundary region pixels of disease in red plane,standard deviation of boundary region pixels of disease in green plane,standard deviation of boundary region pixels of disease in blue plane,standard deviation of core region pixels of disease in red plane,standard deviation of core region pixels of disease in green plane,standard deviation of core region pixels of disease in blue plane
0,196.699448,187.272095,163.721436,255.000000,254.984436,201.214828,5.069962,3.953247,4.509659,22.067413,...,76.622949,0.000000,1.992127,104.030640,35.596849,31.503172,33.609909,45.528642,44.276531,29.145627
1,193.911102,185.048477,159.259232,255.000000,254.972763,195.463943,5.719757,4.595261,5.213928,13.492920,...,78.667365,0.000000,2.635276,107.875634,37.760064,33.921601,36.088317,31.098259,32.017739,16.290980
2,196.202621,189.650909,154.845428,255.000000,255.000000,188.331757,3.657532,2.758713,3.073883,6.208145,...,87.204028,0.000000,0.000000,112.052431,30.319846,26.379182,27.827889,25.772903,24.679760,8.303873
3,174.912750,171.336258,131.140244,250.054550,254.992218,158.674622,6.054382,5.501862,2.354050,3.068024,...,98.298951,35.165782,1.408668,123.630065,38.822828,37.050023,24.387315,13.045190,14.598992,3.795505
4,190.633667,186.197418,143.450989,254.361877,255.000000,181.012802,14.420013,13.163223,12.684631,21.262924,...,90.535645,12.740253,0.000000,115.726531,58.899632,56.421197,55.440789,55.928656,49.215373,26.730121
5,206.348007,197.209167,164.451904,255.000000,254.976654,203.849030,4.618607,4.093323,4.381256,24.815414,...,82.637560,0.000000,2.439810,102.113053,34.006075,32.047496,33.136459,54.103577,48.998853,20.234118
6,181.038834,173.330750,122.635971,254.996109,254.992218,158.176575,12.848053,8.813095,10.174942,45.536530,...,88.793223,0.996086,1.408668,123.754587,55.777962,46.579702,49.910728,67.207328,62.836975,25.956230
7,196.829849,189.182159,156.826355,254.883270,254.984436,191.351166,5.568008,5.069962,4.346237,9.772476,...,88.434884,5.454581,1.992127,110.359769,37.267136,35.596849,33.006071,32.501954,29.051837,13.607809
8,199.943253,191.262787,163.775742,253.789902,254.357986,201.860733,5.622482,4.754791,4.723663,17.094559,...,84.229393,17.524575,12.778939,103.569935,37.444902,34.494401,34.383443,43.330786,39.828743,23.414910
9,185.204590,180.636917,145.798874,255.000000,255.000000,173.584900,4.213943,2.891006,3.704224,4.176926,...,87.147850,0.000000,0.000000,118.879906,32.508434,26.997199,30.509929,19.313778,18.311569,9.447046


In [71]:
df.iloc[0:40]
label = ["Bacterial leaf blight","Brown spot","Leaf smut"]
label_0= []
label_1= []
label_2= []
for i in range(0,40):
    label_0.append(label[0])
for i in range(40,80):
    label_1.append(label[1])
for i in range(80,120):
    label_2.append(label[2])


In [72]:
print(label_0[39])
print(label_1[39])
print(label_2[39])

Bacterial leaf blight
Brown spot
Leaf smut


In [74]:
df["label"] = label_0 + label_1 + label_2

In [75]:
df

,mean of diseased pixels in red plane,mean of diseased pixels in green plane,mean of diseased pixels in blue plane,mean of background pixels in red plane,mean of background pixels in green plane,mean of background pixels in blue plane,mean of boundary region pixels of disease in red plane,mean of boundary region pixels of disease in green plane,mean of boundary region pixels of disease in blue plane,mean of core region pixels of disease in red plane,...,standard deviation of background pixels in red plane,standard deviation of background pixels in green plane,standard deviation of background pixels in blue plane,standard deviation of boundary region pixels of disease in red plane,standard deviation of boundary region pixels of disease in green plane,standard deviation of boundary region pixels of disease in blue plane,standard deviation of core region pixels of disease in red plane,standard deviation of core region pixels of disease in green plane,standard deviation of core region pixels of disease in blue plane,label
0,196.699448,187.272095,163.721436,255.000000,254.984436,201.214828,5.069962,3.953247,4.509659,22.067413,...,0.000000,1.992127,104.030640,35.596849,31.503172,33.609909,45.528642,44.276531,29.145627,Bacterial leaf blight
1,193.911102,185.048477,159.259232,255.000000,254.972763,195.463943,5.719757,4.595261,5.213928,13.492920,...,0.000000,2.635276,107.875634,37.760064,33.921601,36.088317,31.098259,32.017739,16.290980,Bacterial leaf blight
2,196.202621,189.650909,154.845428,255.000000,255.000000,188.331757,3.657532,2.758713,3.073883,6.208145,...,0.000000,0.000000,112.052431,30.319846,26.379182,27.827889,25.772903,24.679760,8.303873,Bacterial leaf blight
3,174.912750,171.336258,131.140244,250.054550,254.992218,158.674622,6.054382,5.501862,2.354050,3.068024,...,35.165782,1.408668,123.630065,38.822828,37.050023,24.387315,13.045190,14.598992,3.795505,Bacterial leaf blight
4,190.633667,186.197418,143.450989,254.361877,255.000000,181.012802,14.420013,13.163223,12.684631,21.262924,...,12.740253,0.000000,115.726531,58.899632,56.421197,55.440789,55.928656,49.215373,26.730121,Bacterial leaf blight
5,206.348007,197.209167,164.451904,255.000000,254.976654,203.849030,4.618607,4.093323,4.381256,24.815414,...,0.000000,2.439810,102.113053,34.006075,32.047496,33.136459,54.103577,48.998853,20.234118,Bacterial leaf blight
6,181.038834,173.330750,122.635971,254.996109,254.992218,158.176575,12.848053,8.813095,10.174942,45.536530,...,0.996086,1.408668,123.754587,55.777962,46.579702,49.910728,67.207328,62.836975,25.956230,Bacterial leaf blight
7,196.829849,189.182159,156.826355,254.883270,254.984436,191.351166,5.568008,5.069962,4.346237,9.772476,...,5.454581,1.992127,110.359769,37.267136,35.596849,33.006071,32.501954,29.051837,13.607809,Bacterial leaf blight
8,199.943253,191.262787,163.775742,253.789902,254.357986,201.860733,5.622482,4.754791,4.723663,17.094559,...,17.524575,12.778939,103.569935,37.444902,34.494401,34.383443,43.330786,39.828743,23.414910,Bacterial leaf blight
9,185.204590,180.636917,145.798874,255.000000,255.000000,173.584900,4.213943,2.891006,3.704224,4.176926,...,0.000000,0.000000,118.879906,32.508434,26.997199,30.509929,19.313778,18.311569,9.447046,Bacterial leaf blight


In [76]:
df.to_csv("Preprocessing_of_agr.csv")